In [1]:
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
from requests.exceptions import HTTPError
from spotipy.exceptions import SpotifyException
import threading
import pandas as pd
import numpy as np
import ast
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from dotenv import load_dotenv
import os


In [1]:
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
from requests.exceptions import HTTPError
from spotipy.exceptions import SpotifyException
import threading
import pandas as pd
import numpy as np
import ast
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from dotenv import load_dotenv
import os


# 스포티파이 api키 입력
client_id = '9138c9fe66ce41dfa7ab1b83ecf6f650'
client_secret = '9521407805044cb7a9573a36519197c1'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



# 스포티파이 위클리 차트 csv파일 저장
class Spotify_weekly_chart:
    def __init__(self, countries, download_directory="/Users/kwonsejin/Desktop/MLops/data/spotify_charts/"): #다운로드 경로는 도커 만들고나서 절대경로로 설정해야함
        self.countries = countries
        self.download_directory = download_directory
        self.options = webdriver.ChromeOptions()
        prefs = {"download.default_directory": self.download_directory}
        self.options.add_experimental_option("prefs", prefs)
        self.driver = None

    def download_charts(self, username, password):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        self.driver.get("https://charts.spotify.com/charts/view/regional-global-weekly/latest")

        # 쿠키 배너 처리
        try:
            accept_cookies = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
            )
            accept_cookies.click()
        except:
            pass

        # 로그인 하기
        login_button = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'Log in'))
        )
        login_button.click()

        id_input = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, 'login-username'))
        )
        id_input.send_keys(username)

        password_input = self.driver.find_element(By.ID, 'login-password')
        password_input.send_keys(password)

        login_button = self.driver.find_element(By.ID, 'login-button')
        login_button.click()

        #로그인 후 로딩대기
        WebDriverWait(self.driver, 10).until(
            EC.url_contains('charts.spotify.com/charts')
        )

        before_files = set(os.listdir(self.download_directory)) #변경 전 파일이름들 확인

        #나라별 200차트 다운로드
        for country in self.countries:
            self.driver.get(f"https://charts.spotify.com/charts/view/regional-{country}-weekly/latest")
            time.sleep(2) # 다운로드 버튼 로딩 대기
            
            csv_download_button = WebDriverWait(self.driver, 60).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-labelledby='csv_download']"))
            )
            csv_download_button.click()
            time.sleep(2)

        # 크롬 셀레늄 끝내기
        if self.driver:
            self.driver.quit()

        #이전파일과 비교해 업데이트된 파일 찾기
        after_files = set(os.listdir(self.download_directory))
        new_files = after_files - before_files
        new_files = list(new_files)

        # 업데이트된 파일들 합치고 저장하기
        new_chart = []
        for file in new_files:
            df = pd.read_csv(f'{self.download_directory + file}')
            new_chart.append(df)

        combined_chart = pd.concat(new_chart, ignore_index=True)
        combined_chart['id'] = combined_chart['uri'].str.split(':').str[2]
        file_name = new_files[0].split('-')[3:]
        file_name = ''.join(file_name[:-1]) + file_name[-1]
        combined_chart.to_csv(f'{self.download_directory}combined_{file_name}', index=False)

        #중복 삭제후 추가된곡만 저장
        exist_songs = pd.read_csv('../data/spotify_songs_new.csv') #도커 올리고 나서 경로수정 해야함
        exist_id = set(exist_songs['track_id'])
        new_songs = combined_chart[~combined_chart['id'].isin(exist_id)]
        new_songs.to_csv('/Users/kwonsejin/Desktop/MLops/data/new_songs.csv', index=False) #도커 올리고 나서 경로수정 해야함




# Rate limit에 맞추어 요청 수 제한
class RateLimiter:
    def __init__(self, max_calls, period):
        self.max_calls = max_calls
        self.period = period
        self.calls = []
        self.lock = threading.Lock()

    def acquire(self):
        with self.lock:
            current_time = time.time()
            # 기간 내의 호출만 남김
            self.calls = [call for call in self.calls if current_time - call < self.period]
            if len(self.calls) >= self.max_calls:
                # 기간이 끝날 때까지 대기
                sleep_time = self.period - (current_time - self.calls[0])
                print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
                time.sleep(sleep_time)
                # 대기 후 호출 리스트 갱신
                self.calls = [call for call in self.calls if time.time() - call < self.period]
            # 호출 기록 추가
            self.calls.append(time.time())



# 새로운 위클리차트 기반으로 데이터베이스 업데이트
class get_info:
    def __init__(self, sp):
        self.sp = sp
        self.ids= pd.read_csv('../data/new_songs.csv')['id'].tolist() #도커 올리고 나서 경로수정 해야함
        self.rate_limiter = RateLimiter(max_calls=5, period=30)

    def chunk_ids(self, chunk_size):
        """IDs를 chunk_size 크기로 분할하는 제너레이터 함수"""
        for i in range(0, len(self.ids), chunk_size):
            yield self.ids[i:i + chunk_size]
    
    def get_audio_features(self):
        audio_features = []
        id_chunks = list(self.chunk_ids(100))
        total_chunks = len(id_chunks)
        for idx, chunk in enumerate(id_chunks):
            self.rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
            try:
                features = self.sp.audio_features(chunk)
                if features is not None:
                    audio_features.extend(features)
                else:
                    print(f"Chunk {idx+1}/{total_chunks}: Received None for features.")
                print(f"Chunk {idx+1}/{total_chunks} processed.")
            except Exception as e:
                print(f"Error fetching audio features for chunk {idx+1}/{total_chunks}: {e}")
        return audio_features
    
    def get_track_info(self):
        track_info = []
        id_chunks = list(self.chunk_ids(50))
        total_chunks = len(id_chunks)
        for idx, chunk in enumerate(id_chunks):
            self.rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
            try:
                response = self.sp.tracks(chunk)
                if response is not None:
                    track_info.extend(response['tracks'])
                else:
                    print(f"Chunk {idx+1}/{total_chunks}: Received None for features.")
                print(f"Chunk {idx+1}/{total_chunks} processed.")
            except Exception as e:
                print(f"Error fetching track_info for chunk {idx+1}/{total_chunks}: {e}")
        return track_info
    
    def fetch(self):
        df = pd.read_csv('../data/spotify_songs_new.csv')
        audio_features = self.get_audio_features()
        features = pd.DataFrame(audio_features)

        track_infos = self.get_track_info()
        infos = pd.DataFrame(track_infos)

        # infos['artists'] = infos['artists'].apply(ast.literal_eval)
        # infos['album'] = infos['album'].apply(ast.literal_eval)    #필요없는듯?

        infos['track_artist'] = infos['artists'].apply(lambda x: ', '.join([artist['name'] for artist in x]))
        infos['track_album_id'] = infos['album'].apply(lambda x: x['id'])
        infos['track_album_name'] = infos['album'].apply(lambda x: x['name'])
        infos['track_album_release_date'] = infos['album'].apply(lambda x: x['release_date'])

        features.rename(columns={'id': 'track_id'}, inplace=True)
        infos.rename(columns={'id': 'track_id', 'name': 'track_name', 'popularity': 'track_popularity'}, inplace=True)

        # 필요한거 골라서 합치기
        df1_selected = features[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                            'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                            'valence', 'tempo', 'duration_ms']]

        df2_selected = infos[['track_id', 'track_name', 'track_artist', 'track_popularity',
                            'track_album_id', 'track_album_name', 'track_album_release_date']]

        df_combined = pd.merge(df1_selected, df2_selected, on='track_id', how='inner')

        # 순서 맞추기
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
        df_combined = df_combined[df.columns]

        # 원본 뒤에 붙이고 저장하기
        df_merged = pd.concat([df, df_combined], ignore_index=True)
        df_merged.drop_duplicates(subset='track_id', inplace=True)
        df_merged.to_csv('../data/spotify_songs_new.csv', index=False)


# 위 클래스들 사용법

# 위클리차트 업데이트
countries = ['kr', 'us', 'global']
downloader = Spotify_weekly_chart(countries)
downloader.download_charts('sejin_kwon@naver.com', 'qykfab-5reZqu-pafhug') #아이디, 패스워드 입력

# 업데이트한 차트 기반으로 데이터베이스 업데이트
spotify_api = get_info(sp)
spotify_api.fetch()

Chunk 1/1 processed.
Chunk 1/1 processed.


KeyError: "['Unnamed: 0'] not in index"

In [ ]:
# 스포티파이 api키 입력
client_id = '9138c9fe66ce41dfa7ab1b83ecf6f650'
client_secret = '9521407805044cb7a9573a36519197c1'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# options = webdriver.ChromeOptions()
# prefs = {"download.default_directory": "./data/spotify_files/"}
# options.add_experimental_option("prefs", prefs)

# # Countries and specific dates
# countries = ['kr', 'us', 'global']

# with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
#     driver.get("https://charts.spotify.com/charts/view/regional-global-weekly/latest")
    
#     # Handle cookie banner if present
#     try:
#         accept_cookies = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
#         )
#         accept_cookies.click()
#     except:
#         pass  # If not present, continue

#     # Wait for the login button and click it
#     login_button = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.LINK_TEXT, 'Log in'))
#     )
#     login_button.click()

#     # Wait for the login form fields
#     id_input = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, 'login-username'))
#     )
#     id_input.send_keys('sejin_kwon@naver.com')

#     password_input = driver.find_element(By.ID, 'login-password')
#     password_input.send_keys('qykfab-5reZqu-pafhug')

#     login_button = driver.find_element(By.ID, 'login-button')
#     login_button.click()

#     # Wait for login to complete
#     WebDriverWait(driver, 10).until(
#         EC.url_contains('charts.spotify.com/charts')
#     )

#     # CSV File Download for specific dates
#     for country in countries:
#         driver.get(f"https://charts.spotify.com/charts/view/regional-{country}-weekly/latest")
#         time.sleep(2)
#         # Wait for the download button to be clickable
#         csv_download_button = WebDriverWait(driver, 60).until(
#             EC.element_to_be_clickable((By.XPATH, "//button[@aria-labelledby='csv_download']"))
#         )
#         csv_download_button.click()

In [6]:

class Spotify_weekly_chart:
    def __init__(self, countries, download_directory="../data/spotify_charts/"): #다운로드 경로는 도커 만들고나서 절대경로로 설정해야함
        self.countries = countries
        self.download_directory = download_directory
        self.options = webdriver.ChromeOptions()
        prefs = {"download.default_directory": self.download_directory}
        self.options.add_experimental_option("prefs", prefs)
        self.driver = None

    def download_charts(self, username, password):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        self.driver.get("https://charts.spotify.com/charts/view/regional-global-weekly/latest")

        # 쿠키 배너 처리
        try:
            accept_cookies = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
            )
            accept_cookies.click()
        except:
            pass

        # 로그인 하기
        login_button = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'Log in'))
        )
        login_button.click()

        id_input = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, 'login-username'))
        )
        id_input.send_keys(username)

        password_input = self.driver.find_element(By.ID, 'login-password')
        password_input.send_keys(password)

        login_button = self.driver.find_element(By.ID, 'login-button')
        login_button.click()

        #로그인 후 로딩대기
        WebDriverWait(self.driver, 10).until(
            EC.url_contains('charts.spotify.com/charts')
        )

        before_files = set(os.listdir(self.download_directory)) #변경 전 파일이름들 확인

        #나라별 200차트 다운로드
        for country in self.countries:
            self.driver.get(f"https://charts.spotify.com/charts/view/regional-{country}-weekly/latest")
            time.sleep(2) # 다운로드 버튼 로딩 대기
            
            csv_download_button = WebDriverWait(self.driver, 60).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-labelledby='csv_download']"))
            )
            csv_download_button.click()
            time.sleep(2)

        # 크롬 셀레늄 끝내기
        if self.driver:
            self.driver.quit()

        #이전파일과 비교해 업데이트된 파일 찾기
        after_files = set(os.listdir(self.download_directory))
        new_files = after_files - before_files
        new_files = list(new_files)

        # 업데이트된 파일들 합치고 저장하기
        new_chart = []
        for file in new_files:
            df = pd.read_csv(f'{self.download_directory + file}')
            new_chart.append(df)

        combined_chart = pd.concat(new_chart, ignore_index=True)
        file_name = new_files[0].split('-')[3:]
        combined_chart.to_csv(f'{self.download_directory}combined_{file_name}')

        #중복 삭제후 추가된곡만 저장
        exist_songs = pd.read_csv('../data/spotify_songs_new.csv') #도커 올리고 나서 경로수정 해야함
        exist_id = set(exist_songs['id'])
        new_songs = combined_chart[~combined_chart['id'].isin(exist_id)]
        new_songs.to_csv('/Users/kwonsejin/Desktop/MLops/data/new_songs.csv', index=False) #도커 올리고 나서 경로수정 해야함


## 위에서 합친 파일 id들이랑 원본 데이터 id들이랑 비교해서 중복 삭제하고 리스트 만들기



In [7]:
# 위 클래스 사용법
countries = ['kr', 'us', 'global']

downloader = Spotify_weekly_chart(countries)

downloader.download_charts('sejin_kwon@naver.com', 'qykfab-5reZqu-pafhug') #아이디, 패스워드 입력

In [6]:
# # 새로운 노래 id들 불러오기
# df = pd.read_csv('../data/new_songs.csv')
# ids = pd.read_csv('../data/new_songs.csv')['id'].tolist()

# # 한번에 호출할 수 있는 100개에 맞추어 자르기
# def chunk_ids(ids, chunk_size=100):
#     for i in range(0, len(ids), chunk_size):
#         yield ids[i:i + chunk_size]

# id_chunks = list(chunk_ids(ids))

In [7]:
# Rate limit에 맞추어 요청 수 제한
class RateLimiter:
    def __init__(self, max_calls, period):
        self.max_calls = max_calls
        self.period = period
        self.calls = []
        self.lock = threading.Lock()

    def acquire(self):
        with self.lock:
            current_time = time.time()
            # 기간 내의 호출만 남김
            self.calls = [call for call in self.calls if current_time - call < self.period]
            if len(self.calls) >= self.max_calls:
                # 기간이 끝날 때까지 대기
                sleep_time = self.period - (current_time - self.calls[0])
                print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
                time.sleep(sleep_time)
                # 대기 후 호출 리스트 갱신
                self.calls = [call for call in self.calls if time.time() - call < self.period]
            # 호출 기록 추가
            self.calls.append(time.time())



rate_limiter = RateLimiter(max_calls=5, period=30)

In [ ]:
class get_info:
    def __init__(self, sp):
        self.sp = sp
        self.ids= pd.read_csv('../data/new_songs.csv')['id'].tolist() #도커 올리고 나서 경로수정 해야함
        self.rate_limiter = RateLimiter(max_calls=5, period=30)

    def chunk_ids(self, chunk_size):
        """IDs를 chunk_size 크기로 분할하는 제너레이터 함수"""
        for i in range(0, len(self.ids), chunk_size):
            yield self.ids[i:i + chunk_size]
    
    def get_audio_features(self):
        audio_features = []
        id_chunks = list(self.chunk_ids(100))
        total_chunks = len(id_chunks)
        for idx, chunk in enumerate(id_chunks):
            self.rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
            try:
                features = self.sp.audio_features(chunk)
                if features is not None:
                    audio_features.extend(features)
                else:
                    print(f"Chunk {idx+1}/{total_chunks}: Received None for features.")
                print(f"Chunk {idx+1}/{total_chunks} processed.")
            except Exception as e:
                print(f"Error fetching audio features for chunk {idx+1}/{total_chunks}: {e}")
        return audio_features
    
    def get_track_info(self):
        track_info = []
        id_chunks = list(self.chunk_ids(50))
        total_chunks = len(id_chunks)
        for idx, chunk in enumerate(id_chunks):
            self.rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
            try:
                response = self.sp.tracks(chunk)
                if response is not None:
                    track_info.extend(response['tracks'])
                else:
                    print(f"Chunk {idx+1}/{total_chunks}: Received None for features.")
                print(f"Chunk {idx+1}/{total_chunks} processed.")
            except Exception as e:
                print(f"Error fetching track_info for chunk {idx+1}/{total_chunks}: {e}")
        return track_info
    
    def fetch(self):
        df = pd.read_csv('../data/spotify_songs_new.csv')
        audio_features = self.get_audio_features()
        features = pd.DataFrame(audio_features)

        track_infos = self.get_track_info()
        infos = pd.DataFrame(track_infos)

        # infos['artists'] = infos['artists'].apply(ast.literal_eval)
        # infos['album'] = infos['album'].apply(ast.literal_eval)    #필요없는듯?

        infos['track_artist'] = infos['artists'].apply(lambda x: ', '.join([artist['name'] for artist in x]))
        infos['track_album_id'] = infos['album'].apply(lambda x: x['id'])
        infos['track_album_name'] = infos['album'].apply(lambda x: x['name'])
        infos['track_album_release_date'] = infos['album'].apply(lambda x: x['release_date'])

        features.rename(columns={'id': 'track_id'}, inplace=True)
        infos.rename(columns={'id': 'track_id', 'name': 'track_name', 'popularity': 'track_popularity'}, inplace=True)

        # 필요한거 골라서 합치기
        df1_selected = features[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                            'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                            'valence', 'tempo', 'duration_ms']]

        df2_selected = infos[['track_id', 'track_name', 'track_artist', 'track_popularity',
                            'track_album_id', 'track_album_name', 'track_album_release_date']]

        df_combined = pd.merge(df1_selected, df2_selected, on='track_id', how='inner')

        # 순서 맞추기
        df_combined = df_combined[df.columns]

        # 원본 뒤에 붙이고 저장하기
        df_merged = pd.concat([df, df_combined], ignore_index=True)
        df_merged.drop_duplicates(subset='track_id', inplace=True)
        df_merged.to_csv('../data/spotify_songs_new.csv', index=False)

In [ ]:
# df = pd.read_csv('../data/spotify_songs.csv')
# features = pd.read_csv('../data/audio_featrues.csv')
# infos = pd.read_csv('../data/track_infos.csv')

# # 필요한 컬럼 생성
# infos['artists'] = infos['artists'].apply(ast.literal_eval)
# infos['album'] = infos['album'].apply(ast.literal_eval)

# infos['track_artist'] = infos['artists'].apply(lambda x: ', '.join([artist['name'] for artist in x]))
# infos['track_album_id'] = infos['album'].apply(lambda x: x['id'])
# infos['track_album_name'] = infos['album'].apply(lambda x: x['name'])
# infos['track_album_release_date'] = infos['album'].apply(lambda x: x['release_date'])

# features.rename(columns={'id': 'track_id'}, inplace=True)
# infos.rename(columns={'id': 'track_id', 'name': 'track_name', 'popularity': 'track_popularity'}, inplace=True)

# # 필요한거 골라서 합치기
# df1_selected = features[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
#                     'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#                     'valence', 'tempo', 'duration_ms']]

# df2_selected = infos[['track_id', 'track_name', 'track_artist', 'track_popularity',
#                     'track_album_id', 'track_album_name', 'track_album_release_date']]

# df_combined = pd.merge(df1_selected, df2_selected, on='track_id', how='inner')

# # 원본데이터 수정
# drop_col = ['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre']
# df.drop(columns=drop_col, inplace=True)

# # 순서 맞추기
# df_combined = df_combined[df.columns]

# # 원본 뒤에 붙이고 저장하기
# df_merged = pd.concat([df, df_combined], ignore_index=True)
# df_merged.to_csv('../data/spotify_songs_new.csv')

# # print(df.shape[0] + df_combined.shape[0])
# # print(df_merged.shape[0])

In [8]:

# # API로 audio features 호출
# def get_audio_features(sp, ids):
#     audio_features = []
#     id_chunks = list(chunk_ids(ids))
#     total_chunks = len(id_chunks)

#     for idx, chunk in enumerate(id_chunks):
#         rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
#         try:
#             features = sp.audio_features(chunk)
#             if features is not None:
#                 audio_features.extend(features)
#             else:
#                 print(f"Chunk {idx+1}/{total_chunks}: Received None for features.")
#             print(f"Chunk {idx+1}/{total_chunks} processed.")
#         except Exception as e:
#             print(f"Error fetching audio features for chunk {idx+1}/{total_chunks}: {e}")
#             # 필요에 따라 재시도 로직 추가 가능
#     return audio_features

In [ ]:
# Audio features 호출
audio_features = get_audio_features(sp, ids)
df = pd.DataFrame(audio_features)

df.to_csv('../data/audio_featrues.csv', index=False)

In [23]:
# df = pd.read_csv('../data/new_songs.csv')

# # 'id' 컬럼만 추출
# ids = df['id'].tolist()

# # 트랙은 50개씩 호출가능
# def chunk_ids(ids, chunk_size=50):
#     for i in range(0, len(ids), chunk_size):
#         yield ids[i:i + chunk_size]

In [24]:
# # API로 audio features 호출
# def get_track_info(sp, ids):
#     track_info = []
#     id_chunks = list(chunk_ids(ids))
#     total_chunks = len(id_chunks)

#     for idx, chunk in enumerate(id_chunks):
#         rate_limiter.acquire()  # 호출 전에 RateLimiter를 통해 제한 관리
#         try:
#             response = sp.tracks(chunk)
#             tracks = response['tracks']
#             if tracks:
#                 track_info.extend(tracks)
#             else:
#                 print(f"Chunk {idx+1}/{total_chunks}: Received empty tracks.")
#             print(f"Chunk {idx+1}/{total_chunks} processed.")
#         except Exception as e:
#             print(f"Error fetching track info for chunk {idx+1}/{total_chunks}: {e}")
#             # 필요에 따라 재시도 로직 추가 가능
#     return track_info


In [ ]:
# Audio features 호출
track_infos = get_track_info(sp, ids)
features = pd.DataFrame(track_infos)

features.to_csv('../data/track_infos.csv', index=False)

In [39]:
df = pd.read_csv('../data/spotify_songs.csv')
features = pd.read_csv('../data/audio_featrues.csv')
infos = pd.read_csv('../data/track_infos.csv')

# 필요한 컬럼 생성
infos['artists'] = infos['artists'].apply(ast.literal_eval)
infos['album'] = infos['album'].apply(ast.literal_eval)

infos['track_artist'] = infos['artists'].apply(lambda x: ', '.join([artist['name'] for artist in x]))
infos['track_album_id'] = infos['album'].apply(lambda x: x['id'])
infos['track_album_name'] = infos['album'].apply(lambda x: x['name'])
infos['track_album_release_date'] = infos['album'].apply(lambda x: x['release_date'])

features.rename(columns={'id': 'track_id'}, inplace=True)
infos.rename(columns={'id': 'track_id', 'name': 'track_name', 'popularity': 'track_popularity'}, inplace=True)

# 필요한거 골라서 합치기
df1_selected = features[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                    'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                    'valence', 'tempo', 'duration_ms']]

df2_selected = infos[['track_id', 'track_name', 'track_artist', 'track_popularity',
                    'track_album_id', 'track_album_name', 'track_album_release_date']]

df_combined = pd.merge(df1_selected, df2_selected, on='track_id', how='inner')

# 원본데이터 수정
drop_col = ['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre']
df.drop(columns=drop_col, inplace=True)

# 순서 맞추기
df_combined = df_combined[df.columns]

# 원본 뒤에 붙이고 저장하기
df_merged = pd.concat([df, df_combined], ignore_index=True)
df_merged.to_csv('../data/spotify_songs_new.csv')

# print(df.shape[0] + df_combined.shape[0])
# print(df_merged.shape[0])

In [34]:
df_combined.columns

Index(['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'track_name', 'track_artist',
       'track_popularity', 'track_album_id', 'track_album_name',
       'track_album_release_date'],
      dtype='object')

In [36]:
drop_col = ['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre']
df.drop(columns=drop_col, inplace=True)

df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In [ ]:
# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# release_date = '2021-01-01'
# query = f'release:{release_date}'
# # song = 'dynamite'
# track_info = sp.search(q=query, type='track')
# track_info

In [ ]:
# import pprint
# pprint.pprint(track_info)

In [ ]:
# class Spotify_audio_features:
#     def __init__(self):

#         client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
#         self.sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#     def get_features(self, song):
#         # get track id information
#         track_info = self.sp.search(q=song, type='track')
#         track_id = track_info["tracks"]["items"][0]["id"]

#         # get audio_feature
#         features = self.sp.audio_features(tracks=[track_id])
#         acousticness = features[0]["acousticness"]
#         danceability = features[0]["danceability"]
#         energy = features[0]["energy"]
#         liveness = features[0]["liveness"]
#         loudness = features[0]["loudness"]
#         valence = features[0]["valence"]
#         mode = features[0]["mode"]

#         result = {"acousticness" : acousticness,
#                     "danceability" : danceability,
#                     "energy" : energy,
#                     "liveness" : liveness,
#                     "loudness" : loudness,
#                     "valence" : valence,
#                     "mode" : mode}

#         return result

# spotify_audio = Spotify_audio_features()

# # get_features 메서드 호출
# song1 = spotify_audio.get_features('dynamite')

# song1

In [ ]:


# auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(auth_manager=auth_manager)

# # API 호출 시 Rate Limiting을 체크하고 처리하는 함수
# def safe_api_call(api_call, *args, **kwargs):
#     while True:
#         try:
#             # Spotify API 호출
#             return api_call(*args, **kwargs)
#         except SpotifyException as e:
#             if e.http_status == 429:
#                 # Retry-After 헤더 값 확인
#                 retry_after = int(e.headers.get("Retry-After", 10))
#                 print(f"Rate limit hit. Retrying after {retry_after} seconds.")
#                 time.sleep(retry_after)  # 지정된 시간만큼 대기 후 다시 시도
#             else:
#                 raise e  # 다른 예외 발생 시 에러 다시 발생
#         except HTTPError as e:
#             print(f"HTTPError occurred: {e}")
#             raise e

# # 특정 날짜에 발매된 트랙을 검색하여 필요한 정보를 DataFrame으로 정리하는 함수
# def search_tracks_by_release_date(sp, release_date):
#     tracks = []
#     limit = 50
#     offset = 0
    
#     while True:
#         # 특정 날짜에 발매된 앨범을 쿼리
#         query = f'release:{release_date}'
#         results = safe_api_call(sp.search, q=query, type='album', limit=limit, offset=offset)
#         albums = results['albums']['items']
        
#         if not albums:
#             break  # 더 이상 결과가 없으면 중단

#         # 각 앨범에서 트랙 정보를 가져오기
#         for album in albums:
#             album_id = album['id']
#             album_name = album['name']
#             album_release_date = album['release_date']

#             # 앨범의 트랙 정보 가져오기
#             album_tracks = safe_api_call(sp.album_tracks, album_id)['items']
            
#             for track in album_tracks:
#                 track_id = track['id']
#                 track_name = track['name']
#                 track_artists = ', '.join([artist['name'] for artist in track['artists']])
#                 track_popularity = safe_api_call(sp.track, track_id)['popularity']

#                 # 트랙의 오디오 피처 (음악 분석 정보)
#                 audio_features = safe_api_call(sp.audio_features, track_id)[0]
                
#                 if audio_features:
#                     # 필요한 정보를 딕셔너리로 저장
#                     track_info = {
#                         'track_id': track_id,
#                         'track_name': track_name,
#                         'track_artist': track_artists,
#                         'track_popularity': track_popularity,
#                         'track_album_id': album_id,
#                         'track_album_name': album_name,
#                         'track_album_release_date': album_release_date,
#                         'playlist_name': None,  # 필요하다면 추가적으로 연관된 Playlist API를 호출해 가져올 수 있음
#                         'playlist_id': None,    # 필요하다면 연관된 Playlist 정보 추가
#                         'playlist_genre': None, # 필요하다면 장르를 가져오도록 수정 가능
#                         'playlist_subgenre': None,
#                         'danceability': audio_features['danceability'],
#                         'energy': audio_features['energy'],
#                         'key': audio_features['key'],
#                         'loudness': audio_features['loudness'],
#                         'mode': audio_features['mode'],
#                         'speechiness': audio_features['speechiness'],
#                         'acousticness': audio_features['acousticness'],
#                         'instrumentalness': audio_features['instrumentalness'],
#                         'liveness': audio_features['liveness'],
#                         'valence': audio_features['valence'],
#                         'tempo': audio_features['tempo'],
#                         'duration_ms': audio_features['duration_ms']
#                     }
#                     tracks.append(track_info)
        
#         offset += limit  # 다음 배치를 가져오기 위해 offset 증가

#     # DataFrame으로 변환
#     return pd.DataFrame(tracks)

# # 발매일 설정 (예: 2020년 2월 2일)
# release_date = '2020-02-02'
# df_tracks = search_tracks_by_release_date(sp, release_date)

# # DataFrame 출력
# print(df_tracks)

In [17]:
# def get_access_token(client_id, client_secret):
#     """스포티파이에서 액세스 토큰을 가져옵니다."""
#     url = 'https://accounts.spotify.com/api/token'
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {'grant_type': 'client_credentials'}

#     while True:
#         try:
#             response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
#             if response.status_code == 429:
#                 retry_after = int(response.headers.get('Retry-After', 10))
#                 print(f"액세스 토큰을 가져오는 중에 레이트 리미트에 도달했습니다. {retry_after}초 후에 다시 시도합니다...")
#                 time.sleep(retry_after)
#                 continue
#             response.raise_for_status()
#             break
#         except requests.exceptions.HTTPError as e:
#             print(f"액세스 토큰 요청 중 에러 발생: {e}")
#             if response.status_code == 429:
#                 retry_after = int(response.headers.get('Retry-After', 10))
#                 print(f"레이트 리미트에 도달했습니다. {retry_after}초 후에 다시 시도합니다...")
#                 time.sleep(retry_after)
#                 continue
#             else:
#                 raise  # 다른 에러는 다시 발생시킵니다.

#     return response.json()['access_token']

# def search_tracks_by_date(access_token, target_date):
#     """특정 날짜에 발매된 트랙을 검색합니다."""
#     url = 'https://api.spotify.com/v1/search'
#     headers = {'Authorization': f'Bearer {access_token}'}
#     query = 'year:2020-02'  # 2020년에 발매된 트랙 검색
#     tracks_on_date = []
#     limit = 50  # 한 번에 가져올 수 있는 최대 아이템 수
#     offset = 0  # 검색 결과의 시작 지점

#     while offset < 1000:  # 최대 1000개의 결과까지 가져올 수 있습니다
#         params = {
#             'q': query,
#             'type': 'track',
#             'limit': limit,
#             'offset': offset
#         }

#         while True:
#             try:
#                 response = requests.get(url, headers=headers, params=params)
#                 if response.status_code == 429:
#                     retry_after = int(response.headers.get('Retry-After', 10))
#                     print(f"레이트 리미트에 도달했습니다. {retry_after}초 후에 다시 시도합니다...")
#                     time.sleep(retry_after)
#                     continue
#                 response.raise_for_status()
#                 break
#             except requests.exceptions.HTTPError as e:
#                 print(f"트랙 검색 중 에러 발생: {e}")
#                 if response.status_code == 429:
#                     retry_after = int(response.headers.get('Retry-After', 10))
#                     print(f"레이트 리미트에 도달했습니다. {retry_after}초 후에 다시 시도합니다...")
#                     time.sleep(retry_after)
#                     continue
#                 else:
#                     raise  # 다른 에러는 다시 발생시킵니다.

#         data = response.json()
#         tracks = data.get('tracks', {}).get('items', [])

#         if not tracks:
#             break  # 더 이상의 결과가 없으면 종료

#         for track in tracks:
#             release_date = track['album']['release_date']
#             release_precision = track['album']['release_date_precision']
#             if release_precision == 'day' and release_date == target_date:
#                 track_info = {
#                     'name': track['name'],
#                     'artists': ', '.join(artist['name'] for artist in track['artists']),
#                     'release_date': release_date
#                 }
#                 tracks_on_date.append(track_info)

#         offset += limit  # 다음 페이지로 이동

#     return tracks_on_date

# def main():
#     access_token = get_access_token(client_id, client_secret)
#     target_date = '2020-02-04'  # 검색할 발매일

#     # 결과를 특정 변수에 저장합니다.
#     search_results = search_tracks_by_date(access_token, target_date)

#     # 필요에 따라 결과를 활용합니다.
#     # 예를 들어, 결과를 출력하려면 아래 코드를 사용하세요.
#     for idx, track in enumerate(search_results, 1):
#         print(f"{idx}. {track['name']} by {track['artists']} (Released on {track['release_date']})")

# if __name__ == "__main__":
#     main()

In [ ]:
# import pprint

# pprint.pprint(search_results)